In [1]:
import sys; sys.path.insert(0, '..')

In [8]:
import geopandas as gpd
import duckdb
from lonboard import Map, ScatterplotLayer
from shapely.geometry import Point
import fiona

gdb_path = "../data/SFCrime.gdb"

# fiona.listlayers(gdb_path)

layers = ["vehicletheft_2013", "vehicletheft_2024"]

mean_layers = {}   # Store ScatterplotLayer objects here

for lyr in layers:
    # Read GeoDataFrame
    gdf = gpd.read_file(gdb_path, layer=lyr)
    
    # Reproject to WGS84 if not already
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # Calculate mean center
    mean_x = gdf.geometry.x.mean()
    mean_y = gdf.geometry.y.mean()

    print(f"{lyr} Mean Center: X = {mean_x:.6f}, Y = {mean_y:.6f}")

    # Make GeoDataFrame for mean center point
    mean_center = gpd.GeoDataFrame(
        {'name': [f'{lyr}_mean_center']},
        geometry=[Point(mean_x, mean_y)],
        crs="EPSG:4326"
    )

    # Create a Scatterplot layer with different colors for each year
    color = [255, 0, 0, 255] if "2013" in lyr else [0, 0, 255, 255]
    
    mean_layer = ScatterplotLayer.from_geopandas(
       mean_center,
       get_fill_color=color,  # Red for 2013, Blue for 2024
       get_radius=500
   )

    mean_layers[lyr] = mean_layer

m = Map(layers=list(mean_layers.values()))
m

vehicletheft_2013 Mean Center: X = -122.430625, Y = 37.756563
vehicletheft_2024 Mean Center: X = -122.424914, Y = 37.758771


In [ ]:
import geopandas as gpd
import duckdb
from lonboard import Map, ScatterplotLayer
from shapely.geometry import Point
import fiona

gdb_path = "../data/SFCrime.gdb"

# fiona.listlayers(gdb_path)

layers = ["vehicletheft_2013", "vehicletheft_2024"]

mean_layers = {}   # Store ScatterplotLayer objects here

for lyr in layers:
    # Read GeoDataFrame
    gdf = gpd.read_file(gdb_path, layer=lyr)

    # Calculate mean center
    mean_x = gdf.geometry.x.mean()
    mean_y = gdf.geometry.y.mean()

    print(f"{lyr} Mean Center: X = {mean_x:.6f}, Y = {mean_y:.6f}")

    # Make GeoDataFrame for mean center point
    mean_center = gpd.GeoDataFrame(
        {'name': [f'{lyr}_mean_center']},
        geometry=[Point(mean_x, mean_y)],
        crs=gdf.crs
    )

    # Create a Scatterplot layer
    mean_layer = ScatterplotLayer.from_geopandas(
       mean_center,
       id=f"mean_center_{lyr}",      # <--- this name appears in legend
       get_fill_color=[0, 255, 0, 255],
       get_radius=500
   )

    mean_layers[lyr] = mean_layer

m = Map(layers=list(mean_layers.values()))

In [11]:
m